In [2]:
# --- Code to Create the Database ---

import mysql.connector

try:
    # Connects to MySQL without specifying a database
    db_connection = mysql.connector.connect(
      host="127.0.0.1",
      user="root",
      password="Akshay@200"
    )
    cursor = db_connection.cursor()

    # This SQL command creates the database if it's missing
    cursor.execute("CREATE DATABASE IF NOT EXISTS phonepe_pulse")
    
    print("Database 'phonepe_pulse' created successfully (or already exists).")
    
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error connecting to MySQL or creating database: {err}")

Database 'phonepe_pulse' created successfully (or already exists).


In [3]:
# --- Inspection Code for Aggregated Transaction ---

# Make sure you have run the cell that defines the 'load_regional' function first.

# 1. Load the data into a DataFrame
df_agg_txn = load_regional("aggregated", "transaction", "state")

# 2. Check if the DataFrame was loaded successfully
if not df_agg_txn.empty:
    print("--- Inspection Results for: aggregated_transaction ---")
    print(f"\nTotal Records Found: {len(df_agg_txn)}")
    print("\nColumns Found:")
    print(list(df_agg_txn.columns))
    print("\nFirst 5 Rows of Data:")
    print(df_agg_txn.head())
    print("\n----------------------------------------------------")
else:
    print("Could not load data for 'aggregated_transaction'. Please check the 'load_regional' function and file paths.")

--- Inspection Results for: aggregated_transaction ---

Total Records Found: 1008

Columns Found:
['from', 'to', 'transactiondata', 'state', 'year', 'quarter', 'date']

First 5 Rows of Data:
            from             to  \
0  1514745000000  1522175400000   
1  1522521000000  1530124200000   
2  1530383400000  1538073000000   
3  1538332200000  1545935400000   
4  1546281000000  1553711400000   

                                     transactiondata  \
0  [{'name': 'Recharge & bill payments', 'payment...   
1  [{'name': 'Recharge & bill payments', 'payment...   
2  [{'name': 'Recharge & bill payments', 'payment...   
3  [{'name': 'Recharge & bill payments', 'payment...   
4  [{'name': 'Recharge & bill payments', 'payment...   

                       state  year  quarter       date  
0  andaman-&-nicobar-islands  2018        1 2018-01-01  
1  andaman-&-nicobar-islands  2018        2 2018-04-01  
2  andaman-&-nicobar-islands  2018        3 2018-07-01  
3  andaman-&-nicobar-islands  201

In [4]:
# --- Create Table Code for Aggregated Transaction ---

import mysql.connector

try:
    db_connection = mysql.connector.connect(
      host="127.0.0.1",
      user="root",
      password="Akshay@200",
      database="phonepe_pulse"
    )
    cursor = db_connection.cursor()

    # This SQL command is built based on our inspection
    create_table_query = """
    CREATE TABLE IF NOT EXISTS aggregated_transaction (
        state VARCHAR(100),
        year INT,
        quarter INT,
        transaction_name VARCHAR(100),
        transaction_count BIGINT,
        transaction_amount DECIMAL(30, 2),
        date DATE
    );
    """
    cursor.execute(create_table_query)
    print("Table 'aggregated_transaction' created successfully (or already exists).")
    
    db_connection.commit()
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error creating table: {err}")

Table 'aggregated_transaction' created successfully (or already exists).


In [5]:
# --- Process and Insert Code for Aggregated Transaction ---

# We already have 'df_agg_txn' from the inspection step.
# This code unpacks the 'transactiondata' column.
processed_rows = []
for index, row in df_agg_txn.iterrows():
    for trans_data in row['transactiondata']:
        payment_instrument = trans_data.get('paymentInstruments', [{}])[0]
        processed_rows.append({
            'state': row['state'],
            'year': row['year'],
            'quarter': row['quarter'],
            'transaction_name': trans_data.get('name'),
            'transaction_count': payment_instrument.get('count'),
            'transaction_amount': payment_instrument.get('amount'),
            'date': row['date']
        })

final_df = pd.DataFrame(processed_rows)

# Now, we insert this final_df into the table
try:
    db_connection = mysql.connector.connect(host="127.0.0.1", user="root", password="Akshay@200", database="phonepe_pulse")
    cursor = db_connection.cursor()
    
    insert_query = "INSERT INTO aggregated_transaction (state, year, quarter, transaction_name, transaction_count, transaction_amount, date) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
    # Convert DataFrame to list of tuples for insertion
    data_to_insert = [tuple(row) for row in final_df.to_numpy()]
    
    cursor.executemany(insert_query, data_to_insert)
    db_connection.commit()
    
    print(f"{cursor.rowcount} records have been successfully inserted into 'aggregated_transaction'.")

except mysql.connector.Error as err:
    print(f"Database error during insertion: {err}")
finally:
    if 'db_connection' in locals() and db_connection.is_connected():
        cursor.close()
        db_connection.close()

5034 records have been successfully inserted into 'aggregated_transaction'.


In [6]:
# --- Inspection Code for Aggregated User ---

# Make sure the 'load_regional' function is defined in your notebook.

# 1. Load the data into a DataFrame
df_agg_user = load_regional("aggregated", "user", "state")

# 2. Check if the DataFrame was loaded successfully
if not df_agg_user.empty:
    print("--- Inspection Results for: aggregated_user ---")
    print(f"\nTotal Records Found: {len(df_agg_user)}")
    print("\nColumns Found:")
    print(list(df_agg_user.columns))
    print("\nFirst 5 Rows of Data:")
    print(df_agg_user.head())
    print("\n-------------------------------------------------")
else:
    print("Could not load data for 'aggregated_user'. Please check the 'load_regional' function and file paths.")

--- Inspection Results for: aggregated_user ---

Total Records Found: 1008

Columns Found:
['aggregated', 'usersbydevice', 'state', 'year', 'quarter', 'date']

First 5 Rows of Data:
                                  aggregated  \
0   {'registeredUsers': 6740, 'appOpens': 0}   
1   {'registeredUsers': 9405, 'appOpens': 0}   
2  {'registeredUsers': 12149, 'appOpens': 0}   
3  {'registeredUsers': 15222, 'appOpens': 0}   
4  {'registeredUsers': 18596, 'appOpens': 0}   

                                       usersbydevice  \
0  [{'brand': 'Xiaomi', 'count': 1665, 'percentag...   
1  [{'brand': 'Xiaomi', 'count': 2303, 'percentag...   
2  [{'brand': 'Xiaomi', 'count': 2950, 'percentag...   
3  [{'brand': 'Xiaomi', 'count': 3719, 'percentag...   
4  [{'brand': 'Xiaomi', 'count': 4576, 'percentag...   

                       state  year  quarter       date  
0  andaman-&-nicobar-islands  2018        1 2018-01-01  
1  andaman-&-nicobar-islands  2018        2 2018-04-01  
2  andaman-&-nicobar-

In [7]:
# --- Create Table Code for Aggregated User ---

import mysql.connector

try:
    db_connection = mysql.connector.connect(
      host="127.0.0.1",
      user="root",
      password="Akshay@200",
      database="phonepe_pulse"
    )
    cursor = db_connection.cursor()

    # This SQL command is built based on our inspection of the 'usersbydevice' column
    create_table_query = """
    CREATE TABLE IF NOT EXISTS aggregated_user (
        state VARCHAR(100),
        year INT,
        quarter INT,
        brand VARCHAR(100),
        brand_count BIGINT,
        brand_percentage DECIMAL(10, 5),
        date DATE
    );
    """
    cursor.execute(create_table_query)
    print("Table 'aggregated_user' created successfully (or already exists).")
    
    db_connection.commit()
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error creating table: {err}")

Table 'aggregated_user' created successfully (or already exists).


In [8]:
# --- Process and Insert Code for Aggregated User ---
import pandas as pd
import mysql.connector

# We use 'df_agg_user' from the inspection step.
# This code unpacks the 'usersbydevice' list.
processed_rows = []
for index, row in df_agg_user.iterrows():
    # Safely handle cases where 'usersbydevice' might be missing or None
    if row['usersbydevice']:
        for device_data in row['usersbydevice']:
            processed_rows.append({
                'state': row.get('state'),
                'year': row.get('year'),
                'quarter': row.get('quarter'),
                'brand': device_data.get('brand'),
                'brand_count': device_data.get('count'),
                'brand_percentage': device_data.get('percentage'),
                'date': row.get('date')
            })

final_df = pd.DataFrame(processed_rows)

# Now, we insert the processed DataFrame into our new table
try:
    db_connection = mysql.connector.connect(host="127.0.0.1", user="root", password="Akshay@200", database="phonepe_pulse")
    cursor = db_connection.cursor()
    
    insert_query = "INSERT INTO aggregated_user (state, year, quarter, brand, brand_count, brand_percentage, date) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
    data_to_insert = [tuple(row) for row in final_df.to_numpy()]
    
    if data_to_insert:
        cursor.executemany(insert_query, data_to_insert)
        db_connection.commit()
        print(f"{cursor.rowcount} records have been successfully inserted into 'aggregated_user'.")
    else:
        print("No processed data was available to insert.")

except mysql.connector.Error as err:
    print(f"Database error during insertion: {err}")
finally:
    if 'db_connection' in locals() and db_connection.is_connected():
        cursor.close()
        db_connection.close()

6732 records have been successfully inserted into 'aggregated_user'.


In [9]:
# --- Inspection Code for Aggregated Insurance ---

# Make sure the 'load_regional' function is defined in your notebook.

# 1. Load the data into a DataFrame
df_agg_ins = load_regional("aggregated", "insurance", "state")

# 2. Check if the DataFrame was loaded successfully
if not df_agg_ins.empty:
    print("--- Inspection Results for: aggregated_insurance ---")
    print(f"\nTotal Records Found: {len(df_agg_ins)}")
    print("\nColumns Found:")
    print(list(df_agg_ins.columns))
    print("\nFirst 5 Rows of Data:")
    print(df_agg_ins.head())
    print("\n----------------------------------------------------")
else:
    print("Could not load data for 'aggregated_insurance'. Please check the 'load_regional' function and file paths.")

--- Inspection Results for: aggregated_insurance ---

Total Records Found: 684

Columns Found:
['from', 'to', 'transactiondata', 'state', 'year', 'quarter', 'date']

First 5 Rows of Data:
            from             to  \
0  1585679400000  1593282600000   
1  1593541800000  1601231400000   
2  1601490600000  1609093800000   
3  1609439400000  1616869800000   
4  1617215400000  1624818600000   

                                     transactiondata  \
0  [{'name': 'Insurance', 'paymentInstruments': [...   
1  [{'name': 'Insurance', 'paymentInstruments': [...   
2  [{'name': 'Insurance', 'paymentInstruments': [...   
3  [{'name': 'Insurance', 'paymentInstruments': [...   
4  [{'name': 'Insurance', 'paymentInstruments': [...   

                       state  year  quarter       date  
0  andaman-&-nicobar-islands  2020        2 2020-04-01  
1  andaman-&-nicobar-islands  2020        3 2020-07-01  
2  andaman-&-nicobar-islands  2020        4 2020-10-01  
3  andaman-&-nicobar-islands  2021  

In [10]:
# --- Create Table Code for Aggregated Insurance ---

import mysql.connector

try:
    db_connection = mysql.connector.connect(
      host="127.0.0.1",
      user="root",
      password="Akshay@200",
      database="phonepe_pulse"
    )
    cursor = db_connection.cursor()

    # This SQL command is built based on our inspection
    create_table_query = """
    CREATE TABLE IF NOT EXISTS aggregated_insurance (
        state VARCHAR(100),
        year INT,
        quarter INT,
        insurance_name VARCHAR(100),
        insurance_count BIGINT,
        insurance_amount DECIMAL(30, 2),
        date DATE
    );
    """
    cursor.execute(create_table_query)
    print("Table 'aggregated_insurance' created successfully (or already exists).")
    
    db_connection.commit()
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error creating table: {err}")

Table 'aggregated_insurance' created successfully (or already exists).


In [11]:
# --- Process and Insert Code for Aggregated Insurance ---
import pandas as pd
import mysql.connector

# We use 'df_agg_ins' from the inspection step.
processed_rows = []
for index, row in df_agg_ins.iterrows():
    # Safely handle cases where 'transactiondata' might be missing
    for ins_data in row.get('transactiondata', []):
        payment_instrument = ins_data.get('paymentInstruments', [{}])[0]
        processed_rows.append({
            'state': row.get('state'),
            'year': row.get('year'),
            'quarter': row.get('quarter'),
            'insurance_name': ins_data.get('name'),
            'insurance_count': payment_instrument.get('count'),
            'insurance_amount': payment_instrument.get('amount'),
            'date': row.get('date')
        })

final_df = pd.DataFrame(processed_rows)

# Now, we insert the processed DataFrame into our new table
try:
    db_connection = mysql.connector.connect(host="127.0.0.1", user="root", password="Akshay@200", database="phonepe_pulse")
    cursor = db_connection.cursor()
    
    insert_query = "INSERT INTO aggregated_insurance (state, year, quarter, insurance_name, insurance_count, insurance_amount, date) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
    data_to_insert = [tuple(row) for row in final_df.to_numpy()]
    
    if data_to_insert:
        cursor.executemany(insert_query, data_to_insert)
        db_connection.commit()
        print(f"{cursor.rowcount} records have been successfully inserted into 'aggregated_insurance'.")
    else:
        print("No processed data was available to insert.")

except mysql.connector.Error as err:
    print(f"Database error during insertion: {err}")
finally:
    if 'db_connection' in locals() and db_connection.is_connected():
        cursor.close()
        db_connection.close()

682 records have been successfully inserted into 'aggregated_insurance'.


In [12]:
# --- Inspection Code for Map Transaction ---

# Make sure the 'load_regional' function is defined in your notebook.

# 1. Load the data into a DataFrame
df_map_txn = load_regional("map", "transaction", "state")

# 2. Check if the DataFrame was loaded successfully
if not df_map_txn.empty:
    print("--- Inspection Results for: map_transaction ---")
    print(f"\nTotal Records Found: {len(df_map_txn)}")
    print("\nColumns Found:")
    print(list(df_map_txn.columns))
    print("\nFirst 5 Rows of Data:")
    print(df_map_txn.head())
    print("\n-------------------------------------------------")
else:
    print("Could not load data for 'map_transaction'. The 'load_regional' function returned an empty DataFrame.")

--- Inspection Results for: map_transaction ---

Total Records Found: 1008

Columns Found:
['hoverdatalist', 'state', 'year', 'quarter', 'date']

First 5 Rows of Data:
                                       hoverdatalist  \
0  [{'name': 'north and middle andaman district',...   
1  [{'name': 'north and middle andaman district',...   
2  [{'name': 'north and middle andaman district',...   
3  [{'name': 'north and middle andaman district',...   
4  [{'name': 'north and middle andaman district',...   

                       state  year  quarter       date  
0  andaman-&-nicobar-islands  2018        1 2018-01-01  
1  andaman-&-nicobar-islands  2018        2 2018-04-01  
2  andaman-&-nicobar-islands  2018        3 2018-07-01  
3  andaman-&-nicobar-islands  2018        4 2018-10-01  
4  andaman-&-nicobar-islands  2019        1 2019-01-01  

-------------------------------------------------


In [13]:
# --- Create Table Code for Map Transaction ---

import mysql.connector

try:
    db_connection = mysql.connector.connect(
      host="127.0.0.1",
      user="root",
      password="Akshay@200",
      database="phonepe_pulse"
    )
    cursor = db_connection.cursor()

    # This SQL command is built based on our inspection of 'hoverdatalist'
    create_table_query = """
    CREATE TABLE IF NOT EXISTS map_transaction (
        state VARCHAR(100),
        year INT,
        quarter INT,
        district_name VARCHAR(100),
        transaction_count BIGINT,
        transaction_amount DECIMAL(30, 2),
        date DATE
    );
    """
    cursor.execute(create_table_query)
    print("Table 'map_transaction' created successfully (or already exists).")
    
    db_connection.commit()
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error creating table: {err}")

Table 'map_transaction' created successfully (or already exists).


In [14]:
# --- Process and Insert Code for Map Transaction ---
import pandas as pd
import mysql.connector

# We use 'df_map_txn' from the inspection step.
processed_rows = []
for index, row in df_map_txn.iterrows():
    # THE FIX: We are now correctly looping through 'hoverdatalist'
    for district_data in row.get('hoverdatalist', []):
        metric = district_data.get('metric', [{}])[0]
        processed_rows.append({
            'state': row.get('state'),
            'year': row.get('year'),
            'quarter': row.get('quarter'),
            'district_name': district_data.get('name'),
            'transaction_count': metric.get('count'),
            'transaction_amount': metric.get('amount'),
            'date': row.get('date')
        })

final_df = pd.DataFrame(processed_rows)

# Now, we insert the processed DataFrame into our new table
try:
    db_connection = mysql.connector.connect(host="127.0.0.1", user="root", password="Akshay@200", database="phonepe_pulse")
    cursor = db_connection.cursor()
    
    insert_query = "INSERT INTO map_transaction (state, year, quarter, district_name, transaction_count, transaction_amount, date) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
    # Drop rows with missing data before inserting
    final_df.dropna(subset=['district_name', 'transaction_count', 'transaction_amount'], inplace=True)
    data_to_insert = [tuple(row) for row in final_df.to_numpy()]
    
    if data_to_insert:
        cursor.executemany(insert_query, data_to_insert)
        db_connection.commit()
        print(f"{cursor.rowcount} records have been successfully inserted into 'map_transaction'.")
    else:
        print("No processed data was available to insert.")

except mysql.connector.Error as err:
    print(f"Database error during insertion: {err}")
finally:
    if 'db_connection' in locals() and db_connection.is_connected():
        cursor.close()
        db_connection.close()

20604 records have been successfully inserted into 'map_transaction'.


In [15]:
# --- Inspection Code for Map User ---

# Make sure the 'load_regional' function is defined in your notebook.

# 1. Load the data into a DataFrame
df_map_user = load_regional("map", "user", "state")

# 2. Check if the DataFrame was loaded successfully
if not df_map_user.empty:
    print("--- Inspection Results for: map_user ---")
    print(f"\nTotal Records Found: {len(df_map_user)}")
    print("\nColumns Found:")
    print(list(df_map_user.columns))
    print("\nFirst 5 Rows of Data:")
    print(df_map_user.head())
    print("\n----------------------------------------------")
else:
    print("Could not load data for 'map_user'. The 'load_regional' function returned an empty DataFrame.")

--- Inspection Results for: map_user ---

Total Records Found: 1008

Columns Found:
['hoverdata', 'state', 'year', 'quarter', 'date']

First 5 Rows of Data:
                                           hoverdata  \
0  {'north and middle andaman district': {'regist...   
1  {'north and middle andaman district': {'regist...   
2  {'north and middle andaman district': {'regist...   
3  {'north and middle andaman district': {'regist...   
4  {'north and middle andaman district': {'regist...   

                       state  year  quarter       date  
0  andaman-&-nicobar-islands  2018        1 2018-01-01  
1  andaman-&-nicobar-islands  2018        2 2018-04-01  
2  andaman-&-nicobar-islands  2018        3 2018-07-01  
3  andaman-&-nicobar-islands  2018        4 2018-10-01  
4  andaman-&-nicobar-islands  2019        1 2019-01-01  

----------------------------------------------


In [16]:
# --- Create Table Code for Map User ---

import mysql.connector

try:
    db_connection = mysql.connector.connect(
      host="127.0.0.1",
      user="root",
      password="Akshay@200",
      database="phonepe_pulse"
    )
    cursor = db_connection.cursor()

    # This SQL command is built based on our inspection of 'hoverdata'
    create_table_query = """
    CREATE TABLE IF NOT EXISTS map_user (
        state VARCHAR(100),
        year INT,
        quarter INT,
        district_name VARCHAR(100),
        registered_users BIGINT,
        app_opens BIGINT,
        date DATE
    );
    """
    cursor.execute(create_table_query)
    print("Table 'map_user' created successfully (or already exists).")
    
    db_connection.commit()
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error creating table: {err}")

Table 'map_user' created successfully (or already exists).


In [17]:
# --- Process and Insert Code for Map User ---
import pandas as pd
import mysql.connector

# We use 'df_map_user' from the inspection step.
processed_rows = []
for index, row in df_map_user.iterrows():
    # THE FIX: We loop through the items of the 'hoverdata' dictionary
    for district_name, metrics in row.get('hoverdata', {}).items():
        processed_rows.append({
            'state': row.get('state'),
            'year': row.get('year'),
            'quarter': row.get('quarter'),
            'district_name': district_name,
            'registered_users': metrics.get('registeredUsers'),
            'app_opens': metrics.get('appOpens'),
            'date': row.get('date')
        })

final_df = pd.DataFrame(processed_rows)

# Now, we insert the processed DataFrame into our new table
try:
    db_connection = mysql.connector.connect(host="127.0.0.1", user="root", password="Akshay@200", database="phonepe_pulse")
    cursor = db_connection.cursor()
    
    insert_query = "INSERT INTO map_user (state, year, quarter, district_name, registered_users, app_opens, date) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
    # Drop rows with missing data before inserting
    final_df.dropna(subset=['district_name', 'registered_users', 'app_opens'], inplace=True)
    data_to_insert = [tuple(row) for row in final_df.to_numpy()]
    
    if data_to_insert:
        cursor.executemany(insert_query, data_to_insert)
        db_connection.commit()
        print(f"{cursor.rowcount} records have been successfully inserted into 'map_user'.")
    else:
        print("No processed data was available to insert.")

except mysql.connector.Error as err:
    print(f"Database error during insertion: {err}")
finally:
    if 'db_connection' in locals() and db_connection.is_connected():
        cursor.close()
        db_connection.close()

20608 records have been successfully inserted into 'map_user'.


In [18]:
# --- Inspection Code for Map Insurance ---

# Make sure the final 'load_regional' function is defined in your notebook.

# 1. Load the data into a DataFrame
df_map_ins = load_regional("map", "insurance", "state")

# 2. Check if the DataFrame was loaded successfully
if not df_map_ins.empty:
    print("--- Inspection Results for: map_insurance ---")
    print(f"\nTotal Records Found: {len(df_map_ins)}")
    print("\nColumns Found:")
    print(list(df_map_ins.columns))
    print("\nFirst 5 Rows of Data:")
    print(df_map_ins.head())
    print("\n-------------------------------------------------")
else:
    print("Could not load data for 'map_insurance'. The 'load_regional' function returned an empty DataFrame.")

--- Inspection Results for: map_insurance ---

Total Records Found: 684

Columns Found:
['meta', 'data', 'state', 'year', 'quarter', 'date']

First 5 Rows of Data:
                                                meta  \
0  {'dataLevel': 'STATE', 'gridLevel': 11, 'perce...   
1  {'dataLevel': 'STATE', 'gridLevel': 11, 'perce...   
2  {'dataLevel': 'STATE', 'gridLevel': 11, 'perce...   
3  {'dataLevel': 'STATE', 'gridLevel': 11, 'perce...   
4  {'dataLevel': 'STATE', 'gridLevel': 11, 'perce...   

                                                data  \
0  {'columns': ['lat', 'lng', 'metric', 'label'],...   
1  {'columns': ['lat', 'lng', 'metric', 'label'],...   
2  {'columns': ['lat', 'lng', 'metric', 'label'],...   
3  {'columns': ['lat', 'lng', 'metric', 'label'],...   
4  {'columns': ['lat', 'lng', 'metric', 'label'],...   

                       state  year  quarter       date  
0  andaman-&-nicobar-islands  2020        2 2020-04-01  
1  andaman-&-nicobar-islands  2020        3 2020

In [33]:
# --- Final Create Table command for map_insurance ---
import mysql.connector

# These variables should already be defined from our previous steps
# db_host = "127.0.0.1"
# db_user = "root"
# db_pass = "Akshay@200"
# db_name = "phonepe_pulse"

try:
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_pass,
        database=db_name
    )
    cursor = db_connection.cursor()

    create_table_query = """
    CREATE TABLE IF NOT EXISTS map_insurance (
        state VARCHAR(100),
        year INT,
        quarter INT,
        district_name VARCHAR(100),
        insurance_count BIGINT,
        insurance_amount DECIMAL(30, 2),
        date DATE
    );
    """
    cursor.execute(create_table_query)
    
    # This line clears any old data from the table
    cursor.execute("TRUNCATE TABLE map_insurance;")
    
    print("Table 'map_insurance' is now ready and empty.")
    
    db_connection.commit()
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error preparing table: {err}")

Table 'map_insurance' is now ready and empty.


In [35]:
# --- Final Process and Insert Code for map_insurance ---
import pandas as pd
import mysql.connector

# This line assumes 'df_map_ins' still exists in your notebook's memory from the inspection step.
processed_rows = []
for index, row in df_map_ins.iterrows():
    # Correctly navigate the nested structure: row -> 'data' (dict) -> 'data' (list of lists)
    for district_list in row.get('data', {}).get('data', []):
        # Safety check: ensure the inner list has enough items
        if len(district_list) > 3:
            # The metric is a single value (count), not a dictionary
            insurance_count = district_list[2] 
            # The district name is the 'label'
            district_name = district_list[3]
            
            processed_rows.append({
                'state': row.get('state'),
                'year': row.get('year'),
                'quarter': row.get('quarter'),
                'district_name': district_name,
                'insurance_count': insurance_count,
                # This data file only provides count, so we set amount to 0
                'insurance_amount': 0, 
                'date': row.get('date')
            })

final_df = pd.DataFrame(processed_rows)

# Now, we insert the processed DataFrame into our table
try:
    # Using the verified variables
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_pass,
        database=db_name
    )
    cursor = db_connection.cursor()
    
    insert_query = "INSERT INTO map_insurance (state, year, quarter, district_name, insurance_count, insurance_amount, date) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
    # Drop rows where essential data might be missing
    final_df.dropna(subset=['district_name', 'insurance_count'], inplace=True)
    data_to_insert = [tuple(row) for row in final_df.to_numpy()]
    
    # --- Using batch insertion to prevent timeouts ---
    batch_size = 1000
    total_inserted = 0
    for i in range(0, len(data_to_insert), batch_size):
        batch = data_to_insert[i:i+batch_size]
        if batch:
            cursor.executemany(insert_query, batch)
            db_connection.commit()
            total_inserted += cursor.rowcount
            
    print(f"SUCCESS: {total_inserted} records have been inserted into 'map_insurance'.")

except mysql.connector.Error as err:
    print(f"DATABASE ERROR during insertion: {err}")
finally:
    if 'db_connection' in locals() and db_connection.is_connected():
        cursor.close()
        db_connection.close()

SUCCESS: 1043137 records have been inserted into 'map_insurance'.


In [21]:
# --- Inspection Code for Top Transaction ---

# Make sure the final 'load_regional' function is defined in your notebook.

# 1. Load the data into a DataFrame
df_top_txn = load_regional("top", "transaction", "state")

# 2. Check if the DataFrame was loaded successfully
if not df_top_txn.empty:
    print("--- Inspection Results for: top_transaction ---")
    print(f"\nTotal Records Found: {len(df_top_txn)}")
    print("\nColumns Found:")
    print(list(df_top_txn.columns))
    print("\nFirst 5 Rows of Data:")
    print(df_top_txn.head())
    print("\n-------------------------------------------------")
else:
    print("Could not load data for 'top_transaction'. The 'load_regional' function returned an empty DataFrame.")

--- Inspection Results for: top_transaction ---

Total Records Found: 1008

Columns Found:
['states', 'districts', 'pincodes', 'state', 'year', 'quarter', 'date']

First 5 Rows of Data:
  states                                          districts  \
0   None  [{'entityName': 'south andaman', 'metric': {'t...   
1   None  [{'entityName': 'south andaman', 'metric': {'t...   
2   None  [{'entityName': 'south andaman', 'metric': {'t...   
3   None  [{'entityName': 'south andaman', 'metric': {'t...   
4   None  [{'entityName': 'south andaman', 'metric': {'t...   

                                            pincodes  \
0  [{'entityName': '744101', 'metric': {'type': '...   
1  [{'entityName': '744101', 'metric': {'type': '...   
2  [{'entityName': '744101', 'metric': {'type': '...   
3  [{'entityName': '744101', 'metric': {'type': '...   
4  [{'entityName': '744103', 'metric': {'type': '...   

                       state  year  quarter       date  
0  andaman-&-nicobar-islands  2018       

In [22]:
# --- Create Table Code for Top Transaction ---

import mysql.connector

try:
    # Using the verified variables from our previous step
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_pass,
        database=db_name
    )
    cursor = db_connection.cursor()

    # This SQL command is built to hold both districts and pincodes
    create_table_query = """
    CREATE TABLE IF NOT EXISTS top_transaction (
        state VARCHAR(100),
        year INT,
        quarter INT,
        entity_name VARCHAR(100),
        entity_type VARCHAR(20),
        transaction_count BIGINT,
        transaction_amount DECIMAL(30, 2),
        date DATE
    );
    """
    cursor.execute(create_table_query)
    print("Table 'top_transaction' created successfully (or already exists).")

    db_connection.commit()
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error creating table: {err}")

Table 'top_transaction' created successfully (or already exists).


In [23]:
# --- Process and Insert Code for Top Transaction ---
import pandas as pd
import mysql.connector

# We use 'df_top_txn' from the inspection step.
processed_rows = []
for index, row in df_top_txn.iterrows():
    # First, process the list of top districts
    for district in row.get('districts', []):
        metric = district.get('metric', {})
        processed_rows.append({
            'state': row.get('state'),
            'year': row.get('year'),
            'quarter': row.get('quarter'),
            'entity_name': district.get('entityName'),
            'entity_type': 'district', # Mark this row as a district
            'transaction_count': metric.get('count'),
            'transaction_amount': metric.get('amount'),
            'date': row.get('date')
        })
    # Next, process the list of top pincodes
    for pincode in row.get('pincodes', []):
        metric = pincode.get('metric', {})
        processed_rows.append({
            'state': row.get('state'),
            'year': row.get('year'),
            'quarter': row.get('quarter'),
            'entity_name': pincode.get('entityName'),
            'entity_type': 'pincode', # Mark this row as a pincode
            'transaction_count': metric.get('count'),
            'transaction_amount': metric.get('amount'),
            'date': row.get('date')
        })

final_df = pd.DataFrame(processed_rows)

# Now, we insert the combined DataFrame into our new table
try:
    # Using the verified variables from our previous step
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_pass,
        database=db_name
    )
    cursor = db_connection.cursor()

    insert_query = "INSERT INTO top_transaction (state, year, quarter, entity_name, entity_type, transaction_count, transaction_amount, date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

    final_df.dropna(subset=['entity_name', 'transaction_count', 'transaction_amount'], inplace=True)
    data_to_insert = [tuple(row) for row in final_df.to_numpy()]

    if data_to_insert:
        cursor.executemany(insert_query, data_to_insert)
        db_connection.commit()
        print(f"{cursor.rowcount} records have been successfully inserted into 'top_transaction'.")
    else:
        print("No processed data was available to insert.")

except mysql.connector.Error as err:
    print(f"Database error during insertion: {err}")
finally:
    if 'db_connection' in locals() and db_connection.is_connected():
        cursor.close()
        db_connection.close()

18293 records have been successfully inserted into 'top_transaction'.


In [24]:
# --- Inspection Code for Top User ---

# Make sure the final 'load_regional' function is defined in your notebook.

# 1. Load the data into a DataFrame
df_top_user = load_regional("top", "user", "state")

# 2. Check if the DataFrame was loaded successfully
if not df_top_user.empty:
    print("--- Inspection Results for: top_user ---")
    print(f"\nTotal Records Found: {len(df_top_user)}")
    print("\nColumns Found:")
    print(list(df_top_user.columns))
    print("\nFirst 5 Rows of Data:")
    print(df_top_user.head())
    print("\n----------------------------------------------")
else:
    print("Could not load data for 'top_user'. The 'load_regional' function returned an empty DataFrame.")

--- Inspection Results for: top_user ---

Total Records Found: 1008

Columns Found:
['states', 'districts', 'pincodes', 'state', 'year', 'quarter', 'date']

First 5 Rows of Data:
  states                                          districts  \
0   None  [{'name': 'south andaman', 'registeredUsers': ...   
1   None  [{'name': 'south andaman', 'registeredUsers': ...   
2   None  [{'name': 'south andaman', 'registeredUsers': ...   
3   None  [{'name': 'south andaman', 'registeredUsers': ...   
4   None  [{'name': 'south andaman', 'registeredUsers': ...   

                                            pincodes  \
0  [{'name': '744103', 'registeredUsers': 1608}, ...   
1  [{'name': '744103', 'registeredUsers': 2188}, ...   
2  [{'name': '744103', 'registeredUsers': 2741}, ...   
3  [{'name': '744103', 'registeredUsers': 3373}, ...   
4  [{'name': '744103', 'registeredUsers': 4136}, ...   

                       state  year  quarter       date  
0  andaman-&-nicobar-islands  2018        1 2018

In [25]:
# --- Create Table Code for Top User ---

import mysql.connector

try:
    # Using the verified variables from our previous step
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_pass,
        database=db_name
    )
    cursor = db_connection.cursor()

    # This SQL command is built to hold both districts and pincodes
    create_table_query = """
    CREATE TABLE IF NOT EXISTS top_user (
        state VARCHAR(100),
        year INT,
        quarter INT,
        entity_name VARCHAR(100),
        entity_type VARCHAR(20),
        registered_users BIGINT,
        date DATE
    );
    """
    cursor.execute(create_table_query)
    print("Table 'top_user' created successfully (or already exists).")

    db_connection.commit()
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error creating table: {err}")

Table 'top_user' created successfully (or already exists).


In [26]:
# --- Process and Insert Code for Top User ---
import pandas as pd
import mysql.connector

# We use 'df_top_user' from the inspection step.
processed_rows = []
for index, row in df_top_user.iterrows():
    # First, process the list of top districts
    for district in row.get('districts', []):
        processed_rows.append({
            'state': row.get('state'),
            'year': row.get('year'),
            'quarter': row.get('quarter'),
            'entity_name': district.get('name'),
            'entity_type': 'district', # Mark this row as a district
            'registered_users': district.get('registeredUsers'),
            'date': row.get('date')
        })
    # Next, process the list of top pincodes
    for pincode in row.get('pincodes', []):
        processed_rows.append({
            'state': row.get('state'),
            'year': row.get('year'),
            'quarter': row.get('quarter'),
            'entity_name': pincode.get('name'),
            'entity_type': 'pincode', # Mark this row as a pincode
            'registered_users': pincode.get('registeredUsers'),
            'date': row.get('date')
        })

final_df = pd.DataFrame(processed_rows)

# Now, we insert the combined DataFrame into our new table
try:
    # Using the verified variables
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_pass,
        database=db_name
    )
    cursor = db_connection.cursor()

    insert_query = "INSERT INTO top_user (state, year, quarter, entity_name, entity_type, registered_users, date) VALUES (%s, %s, %s, %s, %s, %s, %s)"

    final_df.dropna(subset=['entity_name', 'registered_users'], inplace=True)
    data_to_insert = [tuple(row) for row in final_df.to_numpy()]

    if data_to_insert:
        cursor.executemany(insert_query, data_to_insert)
        db_connection.commit()
        print(f"{cursor.rowcount} records have been successfully inserted into 'top_user'.")
    else:
        print("No processed data was available to insert.")

except mysql.connector.Error as err:
    print(f"Database error during insertion: {err}")
finally:
    if 'db_connection' in locals() and db_connection.is_connected():
        cursor.close()
        db_connection.close()

18296 records have been successfully inserted into 'top_user'.


In [27]:
# --- Inspection Code for Top Insurance ---

# Make sure the final 'load_regional' function is defined in your notebook.

# 1. Load the data into a DataFrame
df_top_ins = load_regional("top", "insurance", "state")

# 2. Check if the DataFrame was loaded successfully
if not df_top_ins.empty:
    print("--- Inspection Results for: top_insurance ---")
    print(f"\nTotal Records Found: {len(df_top_ins)}")
    print("\nColumns Found:")
    print(list(df_top_ins.columns))
    print("\nFirst 5 Rows of Data:")
    print(df_top_ins.head())
    print("\n-------------------------------------------------")
else:
    print("Could not load data for 'top_insurance'. The 'load_regional' function returned an empty DataFrame.")

--- Inspection Results for: top_insurance ---

Total Records Found: 684

Columns Found:
['states', 'districts', 'pincodes', 'state', 'year', 'quarter', 'date']

First 5 Rows of Data:
  states                                          districts  \
0   None  [{'entityName': 'nicobars', 'metric': {'type':...   
1   None  [{'entityName': 'south andaman', 'metric': {'t...   
2   None  [{'entityName': 'south andaman', 'metric': {'t...   
3   None  [{'entityName': 'south andaman', 'metric': {'t...   
4   None  [{'entityName': 'south andaman', 'metric': {'t...   

                                            pincodes  \
0  [{'entityName': '744301', 'metric': {'type': '...   
1  [{'entityName': '744112', 'metric': {'type': '...   
2  [{'entityName': '744105', 'metric': {'type': '...   
3  [{'entityName': '744105', 'metric': {'type': '...   
4  [{'entityName': '744103', 'metric': {'type': '...   

                       state  year  quarter       date  
0  andaman-&-nicobar-islands  2020        2 

In [28]:
# --- Create Table Code for Top Insurance ---

import mysql.connector

try:
    # Using the verified variables from our previous step
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_pass,
        database=db_name
    )
    cursor = db_connection.cursor()

    # This SQL command is built to hold both districts and pincodes
    create_table_query = """
    CREATE TABLE IF NOT EXISTS top_insurance (
        state VARCHAR(100),
        year INT,
        quarter INT,
        entity_name VARCHAR(100),
        entity_type VARCHAR(20),
        insurance_count BIGINT,
        insurance_amount DECIMAL(30, 2),
        date DATE
    );
    """
    cursor.execute(create_table_query)
    print("Table 'top_insurance' created successfully (or already exists).")

    db_connection.commit()
    cursor.close()
    db_connection.close()

except mysql.connector.Error as err:
    print(f"Error creating table: {err}")

Table 'top_insurance' created successfully (or already exists).


In [29]:
# --- Process and Insert Code for Top Insurance ---
import pandas as pd
import mysql.connector

# We use 'df_top_ins' from the inspection step.
processed_rows = []
for index, row in df_top_ins.iterrows():
    # First, process the list of top districts
    for district in row.get('districts', []):
        metric = district.get('metric', {})
        processed_rows.append({
            'state': row.get('state'),
            'year': row.get('year'),
            'quarter': row.get('quarter'),
            'entity_name': district.get('entityName'),
            'entity_type': 'district', # Mark this row as a district
            'insurance_count': metric.get('count'),
            'insurance_amount': metric.get('amount'),
            'date': row.get('date')
        })
    # Next, process the list of top pincodes
    for pincode in row.get('pincodes', []):
        metric = pincode.get('metric', {})
        processed_rows.append({
            'state': row.get('state'),
            'year': row.get('year'),
            'quarter': row.get('quarter'),
            'entity_name': pincode.get('entityName'),
            'entity_type': 'pincode', # Mark this row as a pincode
            'insurance_count': metric.get('count'),
            'insurance_amount': metric.get('amount'),
            'date': row.get('date')
        })

final_df = pd.DataFrame(processed_rows)

# Now, we insert the combined DataFrame into our new table
try:
    # Using the verified variables
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_pass,
        database=db_name
    )
    cursor = db_connection.cursor()

    insert_query = "INSERT INTO top_insurance (state, year, quarter, entity_name, entity_type, insurance_count, insurance_amount, date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

    final_df.dropna(subset=['entity_name', 'insurance_count', 'insurance_amount'], inplace=True)
    data_to_insert = [tuple(row) for row in final_df.to_numpy()]

    if data_to_insert:
        cursor.executemany(insert_query, data_to_insert)
        db_connection.commit()
        print(f"{cursor.rowcount} records have been successfully inserted into 'top_insurance'.")
    else:
        print("No processed data was available to insert.")

except mysql.connector.Error as err:
    print(f"Database error during insertion: {err}")
finally:
    if 'db_connection' in locals() and db_connection.is_connected():
        cursor.close()
        db_connection.close()

12273 records have been successfully inserted into 'top_insurance'.
